# packages 

In [1]:
import numpy as np
import pandas as pd
import xarray
import funcs as f
import os

In [2]:
from scipy.interpolate import RectBivariateSpline

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
fs=14
plt.rc("font",size=fs)

### parameters

In [4]:
spacings = {"mynn":333, "ysu":333, "sh":333, "vles":333, "LES":25, "LES_raw":25}

In [5]:
colors = { "LES"   :"k",
          "LES_raw":"k",
          "vles"   :"#808B96",
          "mynn"   :"#641E16",
          "ysu"    :"#A93226",
          "sh"     :"#D98880",}

lss =    { "LES"   :"-",
          "LES_raw":"-",
          "vles"   :"-",
          "mynn"   :"-",
          "ysu"    :"-",
          "sh"     :"-",}

lws =    { "LES"   :2,
          "LES_raw":4,
          "vles"   :2,
          "mynn"   :2,
          "ysu"    :2,
          "sh"     :2}

In [6]:
keys_in_order = ['LES_raw', 'LES', 'vles', 'mynn', 'ysu', 'sh']
sims          = ["LES","vles","mynn","ysu","sh"]
datapath      = "/Volumes/Baby-Orange/XPIA-NCAR/rawest/"
outpath       = "/Volumes/Baby-Orange/XPIA-NCAR/processed/"

### io

In [7]:
days=[20,21,28,29,30]

for iday,day in enumerate(days):
    path = "/Volumes/Baby-Orange/XPIA-NCAR/processed/PBLH_SPATIAL_AVG_15km_by_15km_D02_March{0}.csv".format(day)
    tmp = pd.read_csv(path, index_col=[0], parse_dates=True)
    tmp = tmp['median'].copy()
    if iday==0:
        df_zi = tmp.copy()
    else:
        df_zi = pd.concat([df_zi,tmp],axis=0)

In [8]:
z_zi_target = 0.5

In [9]:
convective      = pd.read_csv(os.path.join(outpath,"datetimes_ziL_gt_25.csv"),header=None,index_col=[0])
convective_list = np.array([ pd.Timestamp(l[0]) for l in convective.values.tolist() ])
nt              = len(convective_list)
print(nt)

269


### bulk

In [14]:
plot_cartesian_psd = False
plot_polar_psd     = False
plot_azimuthal_psd = False
save               = True

varname = 'w'

which_ks = []
which_datetimes = []

year    = 2015
month   = 3
days    = [21,28,29,30]
hours   = np.arange(14,24,1)
minutes = np.arange(0,60,10)

for day in days:
    for hour in hours:
        for minute in minutes:

            datetime     = pd.Timestamp(year=year,month=month,day=day,hour=hour,minute=minute)        
            print("{0:%Y-%m-%d_%H-%M}".format(datetime))

            zi          = df_zi.loc[datetime]
            
            which_datetimes.append(datetime)

            xy_primes, xy_means, k = f.get_prime_and_means(datapath, datetime, zi)
            
            which_ks.append(k)

            f1d    = {}
            psd_2d = {}

            # power and power density in cartesian grid
            for key in keys_in_order:
                print("................")    
                print("Simulation : {0}".format(key))
                print("Spacing : {0}".format(spacings[key]))

                arrays  = xy_primes
                spacing = spacings[key]        
                f1d[key], psd_2d[key] = f.get_psd_2d(arrays[key], spacing=spacing)        

            if plot_cartesian_psd:
                for key in keys_in_order:
                    print(key)    
                    fig = f.plot_psd_cartesian(f1d[key],psd_2d[key],vmin=0,vmax=6)#,log10=False)
                    fig.suptitle(key)

            # power and power density in polar grid
            x         = {}
            y         = {}
            theta     = {}
            r         = {}
            psd_polar = {}

            for key in psd_2d.keys(): 
                x[key], y[key], theta[key], r[key], psd_polar[key] = f.psd_cartesian_to_polar(f1d[key], psd_2d[key])

            if plot_polar_psd:
                for key in keys_in_order:
                    print(key)    
                    fig = f.plot_psd_polar(theta[key],r[key],psd_polar[key])#,log10=False)#,vmin=-5,vmax=7)        

            # power and power density averaged azimuthally
            r_1d   = {}    
            psd_1d = {}

            for key in psd_2d.keys():
                psd_1d[key]   = np.nanmean(psd_polar[key],1)
                r_1d[key]     = np.nanmean(r[key],1)    

            if plot_azimuthal_psd:

                fig = plt.figure(figsize=(6,6))
                ax  = fig.add_subplot(111)
                ax.grid(color='gray', linestyle=':', linewidth=1)

                for key in keys_in_order:
                    ax.loglog(r_1d[key],psd_1d[key],lw=3,label=key.upper(),color=colors[key], linestyle=lss[key], linewidth=lws[key])

                ax.set_xlabel("Wave number $\kappa=\sqrt{\kappa_x^2+\kappa_y^2}$ [1/m]")
                ax.set_ylabel("Azimuthally averaged, two-dimensional\nPower spectrum [m$^2$/s$^2$]")
                ax.set_title("$z/z_i={0}$".format(z_zi_target)+"\n$t=${0:%Y-%m-%d %H:%M} UTC".format(datetime)+"\nvariable: {0}".format(varname))
                plt.legend()

            if save:
                # save everything for this time THIS WONT WORK IF ALL SIMS ARE NOT ON THE SAME GRID, NOT CODED ROBUSTLY ENOUGH
                fout = os.path.join(outpath,"psd_cartesian_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))
                np.savez_compressed(fout, frequency_coarse=f1d["sh"], frequency_fine=f1d["LES"], mynn=psd_2d['mynn'], ysu=psd_2d['ysu'], vles=psd_2d['vles'], sh=psd_2d['sh'], les=psd_2d['LES'], les_raw=psd_2d['LES_raw'])

                fout = os.path.join(outpath,"psd_polar_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))   
                np.savez_compressed(fout, radii_coarse=r["sh"], radii_fine=r["LES"], thetas=theta[key], x_coarse=x["sh"], x_fine=x["LES"], y_coarse=y["sh"], y_fine=y["LES"], mynn=psd_polar['mynn'], ysu=psd_polar['ysu'], vles=psd_polar['vles'], sh=psd_polar['sh'], les=psd_polar['LES'], les_raw=psd_polar['LES_raw'])

                df = pd.DataFrame(index=r_1d["sh"],columns=["sh","mynn","vles","ysu"])
                for key in df.columns:
                    df[key] = psd_1d[key].copy()
                df.columns = [a.lower() for a in df.columns]    
                fout = os.path.join(outpath,"psd_azimuthal_mean_gray_zone_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))               
                df.to_csv(fout)
                
                df = pd.DataFrame(index=r_1d["LES"],columns=["LES","LES_raw"])
                for key in df.columns:
                    df[key] = psd_1d[key].copy()
                df.columns = [a.lower() for a in df.columns]    
                fout = os.path.join(outpath,"psd_azimuthal_mean_les_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
                print("===> {0}".format(fout))               
                df.to_csv(fout)                
                
    fout   = os.path.join(outpath,"k_for_z_zi_0.5_March{0}.csv".format(day))
    series = pd.Series(index=which_datetimes,data=which_ks)
    print("===> {0}".format(fout))                   
    series.to_csv(fout)

2015-03-21_14-00
................
Simulation : LES_raw
Spacing : 25


/Users/pdoubraw/anaconda3/lib/python3.6/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x


................
Simulation : LES
Spacing : 25
................
Simulation : vles
Spacing : 333
................
Simulation : mynn
Spacing : 333
................
Simulation : ysu
Spacing : 333
................
Simulation : sh
Spacing : 333
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_cartesian_2015-03-21_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_polar_2015-03-21_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_azimuthal_mean_gray_zone_2015-03-21_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_azimuthal_mean_les_2015-03-21_14-00_w.npz
2015-03-21_14-10
................
Simulation : LES_raw
Spacing : 25
................
Simulation : LES
Spacing : 25
................
Simulation : vles
Spacing : 333
................
Simulation : mynn
Spacing : 333
................
Simulation : ysu
Spacing : 333
................
Simulation : sh
Spacing : 333
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_cartesian_2015-03-21_14-10_w.npz
===> /Volumes/Bab

/Users/pdoubraw/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:117: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


................
Simulation : LES_raw
Spacing : 25
................
Simulation : LES
Spacing : 25
................
Simulation : vles
Spacing : 333
................
Simulation : mynn
Spacing : 333
................
Simulation : ysu
Spacing : 333
................
Simulation : sh
Spacing : 333
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_cartesian_2015-03-28_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_polar_2015-03-28_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_azimuthal_mean_gray_zone_2015-03-28_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_azimuthal_mean_les_2015-03-28_14-00_w.npz
2015-03-28_14-10
................
Simulation : LES_raw
Spacing : 25
................
Simulation : LES
Spacing : 25
................
Simulation : vles
Spacing : 333
................
Simulation : mynn
Spacing : 333
................
Simulation : ysu
Spacing : 333
................
Simulation : sh
Spacing : 333
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd

# open saved files to make sure they are ok

In [15]:

year     = 2015
month    = 3
day      = 20
hour     = 14
minute   = 0
varname  = 'w'
datetime = pd.Timestamp(year=year,month=month,day=day,hour=hour,minute=minute)
    
# save everything for this time THIS WONT WORK IF ALL SIMS ARE NOT ON THE SAME GRID, NOT ROBUST ENOUGH
fout = os.path.join(outpath,"psd_cartesian_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
print("===> {0}".format(fout))
cartesian = np.load(fout)

fout = os.path.join(outpath,"psd_polar_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
print("===> {0}".format(fout))   
polar = np.load(fout)

fout = os.path.join(outpath,"psd_azimuthal_mean_{0:%Y-%m-%d_%H-%M}_{1}.npz".format(datetime,varname))
print("===> {0}".format(fout))       
df = pd.read_csv(fout, index_col=[0])

===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_cartesian_2015-03-20_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_polar_2015-03-20_14-00_w.npz
===> /Volumes/Baby-Orange/XPIA-NCAR/processed/psd_azimuthal_mean_2015-03-20_14-00_w.npz


### figures (optional)

#### ==> flow field contours

In [21]:
# keys_in_order = ["LES_raw","LES","vles","mynn","ysu","sh"]

# fig = plt.figure(figsize=(6*5,5))
# axs = {}

# for iax,sim in enumerate(arrays.keys()):
#     axs[iax] = fig.add_subplot(1,6,iax+1,aspect="equal")
    
#     p = axs[iax].contourf(arrays[sim], levels=np.arange(-4,4.1,0.2), cmap='magma')
#     variance = np.var(arrays[sim])
#     axs[iax].set_title(sim+"\n $variance={0:.3f}$ m$^2$/s$^2$".format(variance))
#     #plt.colorbar(p)
    
# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.82, 0.255, 0.02, 0.49])
# clb = fig.colorbar(p, cax=cbar_ax, format=FormatStrFormatter('%3.1f'))
# clb.set_label('$w\prime$ [cm/s]', labelpad=15, y=0.45)
# clb.ax.tick_params(direction='in')     

# junkyard

In [17]:
from scipy.integrate import simps

In [18]:
from scipy import fftpack

In [19]:
key="LES_raw"

In [20]:
for key in arrays.keys():
    
    print(".............")
    print(key)
    
    # apply fft
    fftarray  = fftpack.fft2(arrays[key])
    npts,npts = arrays[key].shape
    fr        = fftpack.fftfreq(npts, d=spacings[key])
    var1      = simps(simps(np.abs(fftarray), fr), fr)
    print("{0:.3f} cm2/s2".format(var1*100))


    fftarray_shifted = fftpack.fftshift(fftarray)
    psd_2d           = np.abs(fftarray_shifted)**2
    fr               = np.sort(fr)
    var2             = simps(simps(np.abs(psd_2d), fr), fr)
    print("{0:.3f} cm2/s2".format(var2*100))
    

.............
LES
-11.438 cm2/s2
459369.340 cm2/s2
.............
LES_raw
1902.697 cm2/s2
520327.772 cm2/s2
.............
vles
0.020 cm2/s2
10.484 cm2/s2
.............
mynn
0.004 cm2/s2
10.128 cm2/s2
.............
ysu
0.012 cm2/s2
8.228 cm2/s2
.............
sh
0.012 cm2/s2
8.227 cm2/s2


In [136]:
for key in keys_in_order:
    print (key, simps(simps(psd_2d[key], f1d[key]), f1d[key]))

LES_raw 18.505855486800726
LES 4.224238849404005
vles 3.0188613995547425e-05
mynn 4.363883293651493e-05
ysu 4.224404909962288e-05
sh 4.2233218071212604e-05
